In [ ]:
import os
import pandas as pd
data_dir = "chemin_vers_votre_dossier_NOM_PRENOM"
def add_train_use(data_dir, df=None):
    if df is None:
        df = pd.read_csv(f"{data_dir}/metadata.csv")
    train_files = [f for f in os.listdir(f"{data_dir}/train/malade/") if f[-4:]=='.png']
    train_files.extend(
        [f for f in os.listdir(f"{data_dir}/train/sain/") if f[-4:]=='.png']
    )
    valid_files = [f for f in os.listdir(f"{data_dir}/valid/malade/") if f[-4:]==".png"]
    valid_files.extend(
        [f for f in os.listdir(f"{data_dir}/valid/sain/") if f[-4:]==".png"]
    )
    df["train"] = df["Image Index"].apply(lambda x: x in train_files)
    return df

In [2]:
import pandas as pd
import os
import shutil
def select_data(data_dir, df=None):
    if df is None:
        df = pd.read_csv(f"{data_dir}/metadata.csv")
    patient_ids = df["Patient ID"].unique().tolist()
    select_names = []
    # We look at each patient individually
    for patient in patient_ids:
        df_patient = df[df["Patient ID"]==patient]
        labels = df_patient["Finding Labels"].unique().tolist()
        # We look at all the labels the patients got
        # If he got always the same label we select the first row
        if len(labels)==1:
            select_names.append(df_patient["Image Index"].values[0])
        else:
        # Else we select the first row with a finding
            if "No Finding" in labels:
                labels.remove("No Finding")
            select_names.append(df_patient[df_patient["Finding Labels"]==labels[0]]["Image Index"].values[0])
    print(len(select_names))

    # Create the folders for the selected data
    os.mkdir(f"{data_dir}/selected_data/")
    os.mkdir(f"{data_dir}/selected_data/train/")
    os.mkdir(f"{data_dir}/selected_data/train/malade/")
    os.mkdir(f"{data_dir}/selected_data/train/sain/")
    os.mkdir(f"{data_dir}/selected_data/valid/")
    os.mkdir(f"{data_dir}/selected_data/valid/malade/")
    os.mkdir(f"{data_dir}/selected_data/valid/sain/")

    # Copy only the selected files
    train_malade_files = [f for f in os.listdir(f"{data_dir}/train/malade/") if f[-4:]=='.png']
    train_sain_files = [f for f in os.listdir(f"{data_dir}/train/sain/") if f[-4:]=='.png']
    valid_malade_files = [f for f in os.listdir(f"{data_dir}/valid/malade/") if f[-4:]=='.png']
    valid_sain_files = [f for f in os.listdir(f"{data_dir}/valid/sain/") if f[-4:]=='.png']
    for f in train_malade_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/train/malade/{f}", f"{data_dir}/selected_data/train/malade/")
    for f in train_sain_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/train/sain/{f}", f"{data_dir}/selected_data/train/sain/")
    for f in valid_malade_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/valid/malade/{f}", f"{data_dir}/selected_data/valid/malade/")
    for f in valid_sain_files:
        if f in select_names:
            shutil.copy2(f"{data_dir}/valid/sain/{f}", f"{data_dir}/selected_data/valid/sain/")

    # Save the sub part of the metadata file
    df[df["Image Index"].isin(select_names)].to_csv(f"{data_dir}/selected_data/metadata.csv", index=None)